# Google Colab notebook to fine-tune M2M100

## Dependencies

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
%load_ext autoreload
%autoreload 2
import logging
import os
import random
import torch
import numpy as np
import pandas as pd
import re
import sys
from itertools import cycle
from collections import Counter

In [3]:
#main project directory
project_path="/content/drive/MyDrive/Colab Notebooks/huawei-nlp/final project/"

In [4]:
PATH_TO_DATASET = "/content/drive/MyDrive/Colab Notebooks/huawei-nlp/final project/dataset/"  #Where you stored the dataset

In [5]:
sys.path.append(project_path)

In [6]:
#Path to fairseq scripts
os.environ['PYTHONPATH'] += ":/content/drive/MyDrive/Colab Notebooks/huawei-nlp/final project/low_resource_mt-main/fairseq/"

In [7]:
from cyrillicToLatin import cyrToIPA

In [8]:
!pip install sacremoses
!pip install tensorboardX
%cd /content/drive/MyDrive/Colab Notebooks/huawei-nlp/final project/low_resource_mt-main/fairseq/
!pip install --editable .
!pip install sentencepiece 

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 880 kB 4.1 MB/s 
  Created wheel for sacremoses: filename=sacremoses-0.0.53-py3-none-any.whl size=895260 sha256=3790a5d6945b86110f93c778e17f9f92d274bed22a009791e4c75d009f53e6e0
  Stored in directory: /root/.cache/pip/wheels/82/ab/9b/c15899bf659ba74f623ac776e861cf2eb8608c1825ddec66a4
Successfully built sacremoses
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 125 kB 4.2 MB/s 
/content/drive/MyDrive/Colab Notebooks/huawei-nlp/final project/low_resource_mt-main/fairseq
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Obtaining file:///content/drive/MyDrive/Colab%20Notebooks/huawei-nlp/final%20project/low_resource_mt-main/fairseq
  Installing build dependencies ... done
  Getting requirements to build wheel ... do

## Text preprocessing

In [ ]:
def clean_text(string):
    
    a=re.compile("<.*?>")
    txt=a.sub('',string)
    
    
    if "нет в аудио" in txt:
        txt = txt.replace("[",'')
        txt = txt.replace("]",'')
        txt = txt.replace("нет в аудио",'')
        
    txt = re.sub("[\(\[].*?[\)\]]", "", txt).lower()
    txt = re.sub(r'{[^}]*}','',txt)

    spl = txt.split('/')
    if len(spl) > 2:
      txt = ''.join((spl[0], spl[-1]))
    txt = txt.replace('/','')
    

    txt = txt.replace('*', '')
    txt = txt.replace('"', '')
    txt = txt.replace('«', '')
    txt = txt.replace('»', '')
    txt = txt.replace('ʼ', '')
    txt = txt.replace('—','')
    txt = " ".join(txt.split())
    
    txt = txt.replace('ˊ','')
    txt = txt.replace('ˋ','')

    txt = txt.replace('ѣ','е')
    txt = txt.replace("'",'')
    txt = txt.replace("=",'')
    
    txt = re.sub(r'(?<=[.,])(?=[^\s])', r' ', txt)
    
    txt = re.sub(r'[.,"\'?:!;]', '', txt)
    txt = txt.replace("[",'')
    txt = txt.replace("]",'')
    txt = txt.replace("(",'')
    txt = txt.replace(")",'')
    
    return txt

In [ ]:
#Loading original data
data_csv = pd.read_csv(os.path.join(PATH_TO_DATASET, 'translation.csv')).dropna()

#Remove any possible duplicates
data_csv = data_csv.drop_duplicates(subset=["source", "translation"])
data_csv = data_csv.reset_index()

In [ ]:
data_csv

,index,id,lang,source,translation
0,0,0,kjh,"<?> кӧрзем, ам хыйға кізілер чахсы чуртапча.","когда посмотрю, сейчас умные люди хорошо живут."
1,1,1,xas,"dĭgəttə un abibaʔ, tʼerməndə mĭmbibeʔ.","потом мы муку делали, на мельницу ходили."
2,2,2,xas,dĭgəttə mĭlleʔbəʔjə gijen iʔgö ige sĭre.,"потом они идут туда, где много снего."
3,3,3,evn,горовэ-э нулгӣчэвун.,долго аргишили.
4,4,4,kjh,піреезі алыстырыбызар аны ещё.,кто-нибудь изменит её ещё.
...,...,...,...,...,...
15504,15946,15946,evn,потом.,потом...
15505,15947,15947,xas,sejʔpü dʼala amnobiam.,семь недель жила.
15506,15948,15948,kjh,"ну, анаң мин мынартын… иліг чылларда парыбысха...","ну, потом я отсюда… в пятидесятые годы уехала ..."
15507,15949,15949,evn,ая аhӣ бисо̄н.\n\t\t\t\t,хорошая женщина была.


In [ ]:
#Split into languages
out = {}
for _, string in data_csv.iterrows():


    string1 = clean_text(string['source'])
    string1 = cyrToIPA(string1)
    string2 = clean_text(string['translation'])
    
    if (not string1) or (not string2):
        continue
    
    if string['lang'] not in out:
        out[f"{string['lang']}"] = [string1,string2]
    else:
        out[f"{string['lang']}"].extend([string1,string2])

[весною] голово мōлаңнэрэвун, успеваттиңнэрэвун нāртэдӯ.


In [ ]:
#Writing to languages files
for key in out:
    i=0
    with open(PATH_TO_DATASET+f"{key}.tsv","w",encoding='utf8') as f:
                licycle = cycle(out[key])
                # Prime the pump
                while i<len(out[key])/2:
                    i += 1
                    thiselem = next(licycle)
                    nextelem = next(licycle)
                    f.write(thiselem+"\t"+nextelem+"\n")

## Reading ready files

### Evenki

In [ ]:
evn = pd.read_csv(PATH_TO_DATASET+"evn.tsv", sep="\t", header=None)

In [ ]:
evn = evn.drop_duplicates()
evn=evn.reset_index(drop=True)

In [ ]:
# evn_test_s = evn.sample(frac=0.05).astype(str)
# evn_train_s = evn.drop(index=evn_test_s.index).astype(str)
# evn_test_s=evn_test_s.reset_index(drop=True)
# evn_train_s=evn_train_s.reset_index(drop=True)

In [ ]:
#Split into test and train from original dataset
evn_test = evn.sample(frac=0.15).astype(str)
evn_train = evn.drop(index=evn_test.index).astype(str)
evn_test=evn_test.reset_index(drop=True)
evn_train=evn_train.reset_index(drop=True)

In [ ]:
# evn_train_s.to_csv(PATH_TO_DATASET+'/evn_train_s.txt',sep='\t',index=False,header=False,quoting=None)
# evn_test_s.to_csv(PATH_TO_DATASET+'/evn_test_s.txt',sep='\t',index=False,header=False,quoting=None)

In [ ]:
#Saving original test/train to files
evn_train.to_csv(PATH_TO_DATASET+'/evn_train.txt',sep='\t',index=False,header=False,quoting=None)
evn_test.to_csv(PATH_TO_DATASET+'/evn_test.txt',sep='\t',index=False,header=False,quoting=None)

In [ ]:
#evn_corpus.tsv is taken from Elena Klyachko's corpus (evn.ipynb)
filenames = [PATH_TO_DATASET+"/evn_train.txt", PATH_TO_DATASET+"/evn_corpus.tsv"]
with open(PATH_TO_DATASET+'/new_evn.txt', 'w', encoding='utf8') as outfile:
    for fname in filenames:
        with open(fname, "r", encoding='utf8') as infile:
            for line in infile:
                outfile.write(line)

In [ ]:
evn_train = pd.read_csv(PATH_TO_DATASET+'/new_evn.txt',sep='\t',header=None)
evn_test = pd.read_csv(os.path.join(PATH_TO_DATASET, 'evn_test.txt'),header=None,sep="\t")

In [ ]:
evn_test.columns=['input_text','target_text']
evn_train.columns=['input_text','target_text']

In [ ]:
evn_all = pd.concat([evn_train,evn_test],axis=0)

In [ ]:
result0 = Counter(" ".join(evn_all['input_text'].values.tolist()).split(" ")).items()
result1 = Counter(" ".join(evn_all['target_text'].values.tolist()).split(" ")).items()

In [ ]:
evn_train = evn_train.drop_duplicates()
evn_train=evn_train.reset_index(drop=True)

In [ ]:
#Removing duplicates with test set
new_train = evn_train.copy()
for index1, row1 in evn_test.iterrows():
  for index2, row2 in evn_train.iterrows():
    if row1[0] == row2[0] or row1[1] == row2[1]:
      new_train = new_train.drop([index1])
      break

In [ ]:
evn_train = new_train.copy()

In [ ]:
# #Lowercase and remove trailing spaces
# evn_train["input_text"] = evn_train.apply(lambda x: x.input_text.strip(), axis=1)
# evn_train["target_text"] = evn_train.target_text.apply(lambda x: x.lower())

In [ ]:
#Train / Validation ~ 200 sents Split
#evn_validation = evn_train.sample(frac=0.022).astype(str)
evn_validation = evn_train.sample(frac=0.03).astype(str)
evn_train = evn_train.drop(index=evn_validation.index).astype(str)

In [ ]:
# #Train / Validation ~200 sents Split
# evn_validation_s = evn_train_s.sample(frac=0.0527).astype(str)
# evn_train_s = evn_train_s.drop(index=evn_validation_s.index).astype(str)

In [ ]:
evn_train=evn_train.reset_index(drop=True)
evn_validation=evn_validation.reset_index(drop=True)

In [ ]:
#Saving full train/val set to files
evn_train.to_csv(PATH_TO_DATASET+'/evn_train_f.txt',sep='\t',index=False,header=False,quoting=None)
evn_validation.to_csv(PATH_TO_DATASET+'/evn_val_f.txt',sep='\t',index=False,header=False,quoting=None)

In [ ]:
# evn_train_s=evn_train_s.reset_index(drop=True)
# evn_validation_s=evn_validation_s.reset_index(drop=True)

### Kamas

In [ ]:
xas = pd.read_csv(PATH_TO_DATASET+"/xas.tsv", sep="\t", header=None)

In [ ]:
xas = xas.drop_duplicates()

In [ ]:
xas=xas.reset_index(drop=True)

In [ ]:
#Split into test and train from original dataset
xas_test = xas.sample(frac=0.045).astype(str)
xas_train = xas.drop(index=xas_test.index).astype(str)
xas_test=xas_test.reset_index(drop=True)
xas_train=xas_train.reset_index(drop=True)

In [ ]:
#Saving original test/train to files
xas_train.to_csv(PATH_TO_DATASET+'/xas_train.txt',sep='\t',index=False,header=False,quoting=None)
xas_test.to_csv(PATH_TO_DATASET+'/xas_test.txt',sep='\t',index=False,header=False,quoting=None)

In [ ]:
#xas-inel.tsv is taken from Hamburg University corpus (inel.ipynb)
filenames = [PATH_TO_DATASET+"/xas_train.txt", PATH_TO_DATASET+"/xas-inel.tsv"]
with open(PATH_TO_DATASET+'/new_xas.txt', 'w', encoding='utf8') as outfile:
    for fname in filenames:
        with open(fname, "r", encoding='utf8') as infile:
            for line in infile:
                outfile.write(line)

In [ ]:
xas_train = pd.read_csv(PATH_TO_DATASET+'/new_xas.txt',sep='\t',header=None)
xas_test = pd.read_csv(os.path.join(PATH_TO_DATASET, 'xas_test.txt'),header=None,sep="\t")

In [ ]:
xas_test.columns=['input_text','target_text']
xas_train.columns=['input_text','target_text']

In [ ]:
xas_train = xas_train.drop_duplicates()
xas_train=xas_train.reset_index(drop=True)

In [ ]:
#Removing duplicates with test set
new_train = xas_train.copy()
for index1, row1 in xas_test.iterrows():
  for index2, row2 in xas_train.iterrows():
    if row1[0] == row2[0] or row1[1] == row2[1]:
      new_train = new_train.drop([index1])
      break

In [ ]:
new_train=new_train.reset_index(drop=True)
xas_train = new_train.copy()

In [ ]:
xas_all = pd.concat([xas_train,xas_test],axis=0)

In [ ]:
result0 = Counter(" ".join(xas_all['input_text'].values.tolist()).split(" ")).items()
result1 = Counter(" ".join(xas_all['target_text'].values.tolist()).split(" ")).items()

6054

In [ ]:
xas_train.columns = ["input_text", "target_text"]

In [ ]:
xas_train

,input_text,target_text
0,bar ĭmbi nuldelaʔbeʔje,ставят все
1,măn măndem unna amnolam bar,я говорю одна буду жить ведь
2,tüshelde măna maktanerzitte tănzi,научи меня восхвалять тебя
3,dĭ shobi surarlaʔbe ĭmbi tănan kereʔ,тот пришел спрашивает что тебе надо
4,ibi ej mobi,схватил… не смог
...,...,...
13703,iʔgo orengeti iʔbeleʔ maʔlaʔ kojobiam,от большого рода я один остался
13704,tuganbegeti turzoleʔ kojobiam,от близких я отбился
13705,kijen kolajlaʔbiam tuzangbe maʔlaʔ kojobiiʔ,там где я рыбу ловил озера мои остались
13706,tuje dizengbe em kuʔ,не видать их мне больше


In [ ]:
#Train / Validation ~200 sents Split
xas_validation = xas_train.sample(frac=0.016).astype(str)
xas_train = xas_train.drop(index=xas_validation.index).astype(str)

In [ ]:
xas_train=xas_train.reset_index(drop=True)
xas_validation=xas_validation.reset_index(drop=True)

In [ ]:
#Saving full train/val set to files
xas_train.to_csv(PATH_TO_DATASET+'/xas_train_f.txt',sep='\t',index=False,header=False,quoting=None)
xas_validation.to_csv(PATH_TO_DATASET+'/xas_val_f.txt',sep='\t',index=False,header=False,quoting=None)

### Mari

In [ ]:
mhr = pd.read_csv(PATH_TO_DATASET+"/mhr.tsv", sep="\t", header=None)

In [ ]:
mhr = mhr.drop_duplicates()

In [ ]:
mhr=mhr.reset_index(drop=True)

In [ ]:
#Split into test and train from original dataset
mhr_test = mhr.sample(frac=0.09).astype(str)
mhr_train = mhr.drop(index=mhr_test.index).astype(str)
mhr_test=mhr_test.reset_index(drop=True)
mhr_train=mhr_train.reset_index(drop=True)

In [ ]:
#Saving original test/train to files
mhr_train.to_csv(PATH_TO_DATASET+'/mhr_train.txt',sep='\t',index=False,header=False,quoting=None)
mhr_test.to_csv(PATH_TO_DATASET+'/mhr_test.txt',sep='\t',index=False,header=False,quoting=None)

In [ ]:
#xas-inel.tsv is taken from Hamburg University corpus (inel.ipynb)
filenames = [PATH_TO_DATASET+"/mhr_train.txt", PATH_TO_DATASET+"/mhr_corpus.tsv"]
with open(PATH_TO_DATASET+'/new_mhr.txt', 'w', encoding='utf8') as outfile:
    for fname in filenames:
        with open(fname, "r", encoding='utf8') as infile:
            for line in infile:
                outfile.write(line)

In [ ]:
mhr_train = pd.read_csv(PATH_TO_DATASET+'/new_mhr.txt',sep='\t',header=None)

In [ ]:
mhr_test.columns=['input_text','target_text']
mhr_train.columns=['input_text','target_text']

In [ ]:
mhr_train

,input_text,target_text
0,kok rush udyr-shamych ulyt yle,две русские девушки были
1,wara malyzhe,пусть поспит
2,ere kolysho imnetlan da mo da pesh shortynam da,плакала из-за павшей лошади и прочего
3,myj kuznechysh koshtynam,она не отсюда потому что
4,izirak ergymyn kok ik ergyzhe ulo,а у младшего один сын
...,...,...
109735,myn ynem uzh ti kartinym jasht,я совсем не хочу видеть этой картины
109736,tamahan kostan pi terwenem kechaltat pel mongy...,какая-то злая собака набросилась на меня и сов...
109737,iwanna jashtok li wet pajan sudjan yshtysh kid...,наш иван пропал ведь сделала рука богатого суд...
109738,pesh kuleshan godym gyna kutyra molo saltak-wl...,эрнест говорит только когда очень необходимо д...


In [ ]:
#Calculating vocab size
mhr_all = pd.concat([mhr_train,mhr_test],axis=0)
result0 = Counter(" ".join(mhr_all['input_text'].values.tolist()).split(" ")).items()
result1 = Counter(" ".join(mhr_all['target_text'].values.tolist()).split(" ")).items()
len(result1)

In [ ]:
len(result0)

94268

In [ ]:
mhr_train = mhr_train.drop_duplicates()
mhr_train=mhr_train.reset_index(drop=True)

In [ ]:
mhr_train

,input_text,target_text
0,kok rush udyr-shamych ulyt yle,две русские девушки были
1,wara malyzhe,пусть поспит
2,ere kolysho imnetlan da mo da pesh shortynam da,плакала из-за павшей лошади и прочего
3,myj kuznechysh koshtynam,она не отсюда потому что
4,izirak ergymyn kok ik ergyzhe ulo,а у младшего один сын
...,...,...
109100,myn ynem uzh ti kartinym jasht,я совсем не хочу видеть этой картины
109101,tamahan kostan pi terwenem kechaltat pel mongy...,какая-то злая собака набросилась на меня и сов...
109102,iwanna jashtok li wet pajan sudjan yshtysh kid...,наш иван пропал ведь сделала рука богатого суд...
109103,pesh kuleshan godym gyna kutyra molo saltak-wl...,эрнест говорит только когда очень необходимо д...


In [ ]:
#Removing duplicates with test set
new_train = mhr_train.copy()
for index1, row1 in mhr_test.iterrows():
  for index2, row2 in mhr_train.iterrows():
    if row1[0] == row2[0] or row1[1] == row2[1]:
      new_train = new_train.drop([index1])
      break

In [ ]:
new_train=new_train.reset_index(drop=True)

In [ ]:
len(result0)

7294

In [ ]:
len(result1)

6054

In [ ]:
#evn_train=new_train.dropna()

In [ ]:
mhr_train = new_train.copy()

In [ ]:
mhr_train.columns = ["input_text", "target_text"]

In [ ]:
#Train / Validation ~200 sents Split
mhr_validation = mhr_train.sample(frac=0.002).astype(str)
mhr_train = mhr_train.drop(index=mhr_validation.index).astype(str)

In [ ]:
mhr_train=mhr_train.reset_index(drop=True)
mhr_validation=mhr_validation.reset_index(drop=True)

In [ ]:
mhr_train_s=mhr_train_s.reset_index(drop=True)
mhr_validation_s=mhr_validation_s.reset_index(drop=True)

In [ ]:
#Saving full train/val set to files
mhr_train.to_csv(PATH_TO_DATASET+'/mhr_train_f.txt',sep='\t',index=False,header=False,quoting=None)
mhr_validation.to_csv(PATH_TO_DATASET+'/mhr_val_f.txt',sep='\t',index=False,header=False,quoting=None)

In [ ]:
mhr_validation

,input_text,target_text
0,tyge kok shagat cholo kajymek kuryk chongash k...,проехав так около двух часов мы поднялись на в...
1,shudo utla jen yle a kyzyte,когда я приехала более ста человек а сейчас
2,shuwyrlum kombo ale shoryk jolwurgyn izi luzho...,трубочку для надувания пузыря изготовляют из м...
3,urzha ozym,озимь ржи
4,toj dene yshtyme shyga shortnyla kojyn cholgyz...,латунное кольцо на поясе проски блестит как зо...
...,...,...
213,tukydymo mlandym mushash gaj ogyl,нетронутой земли кажется не найти
214,knigam ynem pu,не хочу отдавать книгу
215,shudo otyl,стерня от скошенной травы
216,tushka dene kashtalaltash,собираться группами в стаи


## Change current directory to run scripts

In [9]:
%cd /content/drive/MyDrive/Colab Notebooks/huawei-nlp/final project/dataset

/content/drive/MyDrive/Colab Notebooks/huawei-nlp/final project/dataset


## Preparing for fine-tuning

In [ ]:
#Download  pretrained model, vocabulary and tokenizer
!wget -qq "https://dl.fbaipublicfiles.com/m2m_100/spm.128k.model"
!wget -qq "https://dl.fbaipublicfiles.com/m2m_100/data_dict.128k.txt"
!wget -qq "https://dl.fbaipublicfiles.com/m2m_100/model_dict.128k.txt"
!wget -qq "https://dl.fbaipublicfiles.com/m2m_100/language_pairs_small_models.txt"
!wget "https://dl.fbaipublicfiles.com/m2m_100/418M_last_checkpoint.pt"

### Evenki

In [ ]:
#Saving files for the model
train_txt = "\n".join(evn_train.input_text.values.tolist())
file = open(PATH_TO_DATASET+"/evenki_txt_train.txt", "w")
file.write(train_txt)
file.close()


train_target_txt = "\n".join(evn_train.target_text.values.tolist())
file = open(PATH_TO_DATASET+"/russian_txt_train.txt", "w")
file.write(train_target_txt)
file.close()

evn_validation_txt = "\n".join(evn_validation.input_text.values.tolist())
file = open(PATH_TO_DATASET+"/evenki_txt_validation.txt", "w")
file.write(evn_validation_txt)
file.close()


evn_validation_target_txt = "\n".join(evn_validation.target_text.values.tolist())
file = open(PATH_TO_DATASET+"/russian_txt_validation.txt", "w")
file.write(evn_validation_target_txt)
file.close()

In [ ]:
train_txt = "\n".join(evn_train_s.input_text.values.tolist())
file = open(PATH_TO_DATASET+"/evenki_txt_train_s.txt", "w")
file.write(train_txt)
file.close()


train_target_txt = "\n".join(evn_train_s.target_text.values.tolist())
file = open(PATH_TO_DATASET+"/russian_txt_train_s.txt", "w")
file.write(train_target_txt)
file.close()

evn_validation_txt = "\n".join(evn_validation_s.input_text.values.tolist())
file = open(PATH_TO_DATASET+"/evenki_txt_validation_s.txt", "w")
file.write(evn_validation_txt)
file.close()


evn_validation_target_txt = "\n".join(evn_validation_s.target_text.values.tolist())
file = open(PATH_TO_DATASET+"/russian_txt_validation_s.txt", "w")
file.write(evn_validation_target_txt)
file.close()

In [ ]:
#Tokenize text
!python ../low_resource_mt-main/fairseq/scripts/spm_encode.py \
        --model spm.128k.model \
        --output_format=piece \
        --inputs=evenki_txt_train.txt \
        --outputs=train.evn
        
!python ../low_resource_mt-main/fairseq/scripts/spm_encode.py \
        --model spm.128k.model \
        --output_format=piece \
        --inputs=russian_txt_train.txt \
        --outputs=train.ru
        
!python ../low_resource_mt-main/fairseq/scripts/spm_encode.py \
        --model spm.128k.model \
        --output_format=piece \
        --inputs=evenki_txt_validation.txt \
        --outputs=val.evn
        
!python ../low_resource_mt-main/fairseq/scripts/spm_encode.py \
        --model spm.128k.model \
        --output_format=piece \
        --inputs=russian_txt_validation.txt \
        --outputs=val.ru
 

skipped 0 empty lines
filtered 0 lines
skipped 0 empty lines
filtered 0 lines
skipped 0 empty lines
filtered 0 lines
skipped 0 empty lines
filtered 0 lines


In [ ]:
#Tokenize text
!python ../low_resource_mt-main/fairseq/scripts/spm_encode.py \
        --model spm.128k.model \
        --output_format=piece \
        --inputs=evenki_txt_train_s.txt \
        --outputs=train_s.evn
        
!python ../low_resource_mt-main/fairseq/scripts/spm_encode.py \
        --model spm.128k.model \
        --output_format=piece \
        --inputs=russian_txt_train_s.txt \
        --outputs=train_s.ru
        
!python ../low_resource_mt-main/fairseq/scripts/spm_encode.py \
        --model spm.128k.model \
        --output_format=piece \
        --inputs=evenki_txt_validation_s.txt \
        --outputs=val_s.evn
        
!python ../low_resource_mt-main/fairseq/scripts/spm_encode.py \
        --model spm.128k.model \
        --output_format=piece \
        --inputs=russian_txt_validation_s.txt \
        --outputs=val_s.ru
 

skipped 0 empty lines
filtered 0 lines
skipped 0 empty lines
filtered 0 lines
skipped 0 empty lines
filtered 0 lines
skipped 0 empty lines
filtered 0 lines


In [ ]:
# length ratio cleaning
!perl clean-corpus-n.perl --ratio 3 train evn ru spm_encode/train 1 250
!perl clean-corpus-n.perl --ratio 3 val evn ru spm_encode/val 1 250

clean-corpus.perl: processing train.evn & .ru to spm_encode/train, cutoff 1-250, ratio 3

Input sentences: 6816  Output sentences:  6768
clean-corpus.perl: processing val.evn & .ru to spm_encode/val, cutoff 1-250, ratio 3

Input sentences: 211  Output sentences:  210


In [ ]:
# !perl clean-corpus-n.perl --ratio 3 train_s evn ru spm_encode/train_s 1 250
# !perl clean-corpus-n.perl --ratio 3 val_s evn ru spm_encode/val_s 1 250

### Kamas

In [ ]:
#Saving files for the model
train_txt = "\n".join(xas_train.input_text.values.tolist())
file = open(PATH_TO_DATASET+"/xas_txt_train.txt", "w")
file.write(train_txt)
file.close()


train_target_txt = "\n".join(xas_train.target_text.values.tolist())
file = open(PATH_TO_DATASET+"/xas_russian_txt_train.txt", "w")
file.write(train_target_txt)
file.close()

xas_validation_txt = "\n".join(xas_validation.input_text.values.tolist())
file = open(PATH_TO_DATASET+"/xas_txt_validation.txt", "w")
file.write(xas_validation_txt)
file.close()


xas_validation_target_txt = "\n".join(xas_validation.target_text.values.tolist())
file = open(PATH_TO_DATASET+"/xas_russian_txt_validation.txt", "w")
file.write(xas_validation_target_txt)
file.close()

In [ ]:
#Tokenize text
!python ../low_resource_mt-main/fairseq/scripts/spm_encode.py \
        --model spm.128k.model \
        --output_format=piece \
        --inputs=xas_txt_train.txt \
        --outputs=train.xas
        
!python ../low_resource_mt-main/fairseq/scripts/spm_encode.py \
        --model spm.128k.model \
        --output_format=piece \
        --inputs=xas_russian_txt_train.txt \
        --outputs=train.ru
        
!python ../low_resource_mt-main/fairseq/scripts/spm_encode.py \
        --model spm.128k.model \
        --output_format=piece \
        --inputs=xas_txt_validation.txt \
        --outputs=val.xas
        
!python ../low_resource_mt-main/fairseq/scripts/spm_encode.py \
        --model spm.128k.model \
        --output_format=piece \
        --inputs=xas_russian_txt_validation.txt \
        --outputs=val.ru
 

processed 10000 lines
skipped 0 empty lines
filtered 0 lines
processed 10000 lines
skipped 0 empty lines
filtered 0 lines
skipped 0 empty lines
filtered 0 lines
skipped 0 empty lines
filtered 0 lines


In [ ]:
# length ratio cleaning
!perl clean-corpus-n.perl --ratio 3 train xas ru spm_encode/train 1 250
!perl clean-corpus-n.perl --ratio 3 val xas ru spm_encode/val 1 250

clean-corpus.perl: processing train.xas & .ru to spm_encode/train, cutoff 1-250, ratio 3
.
Input sentences: 13489  Output sentences:  13210
clean-corpus.perl: processing val.xas & .ru to spm_encode/val, cutoff 1-250, ratio 3

Input sentences: 219  Output sentences:  210


### Mari

In [ ]:
#Saving files for the model
train_txt = "\n".join(mhr_train.input_text.values.tolist())
file = open(PATH_TO_DATASET+"/mari_txt_train.txt", "w")
file.write(train_txt)
file.close()


train_target_txt = "\n".join(mhr_train.target_text.values.tolist())
file = open(PATH_TO_DATASET+"/mari_russian_txt_train.txt", "w")
file.write(train_target_txt)
file.close()

mhr_validation_txt = "\n".join(mhr_validation.input_text.values.tolist())
file = open(PATH_TO_DATASET+"/mari_txt_validation.txt", "w")
file.write(mhr_validation_txt)
file.close()


mhr_validation_target_txt = "\n".join(mhr_validation.target_text.values.tolist())
file = open(PATH_TO_DATASET+"/mari_russian_txt_validation.txt", "w")
file.write(mhr_validation_target_txt)
file.close()

In [ ]:
#Tokenize text
!python ../low_resource_mt-main/fairseq/scripts/spm_encode.py \
        --model spm.128k.model \
        --output_format=piece \
        --inputs=mari_txt_train.txt \
        --outputs=train.mhr
        
!python ../low_resource_mt-main/fairseq/scripts/spm_encode.py \
        --model spm.128k.model \
        --output_format=piece \
        --inputs=mari_russian_txt_train.txt \
        --outputs=train.ru
        
!python ../low_resource_mt-main/fairseq/scripts/spm_encode.py \
        --model spm.128k.model \
        --output_format=piece \
        --inputs=mari_txt_validation.txt \
        --outputs=val.mhr
        
!python ../low_resource_mt-main/fairseq/scripts/spm_encode.py \
        --model spm.128k.model \
        --output_format=piece \
        --inputs=mari_russian_txt_validation.txt \
        --outputs=val.ru
 

processed 10000 lines
processed 20000 lines
processed 30000 lines
processed 40000 lines
processed 50000 lines
processed 60000 lines
processed 70000 lines
processed 80000 lines
processed 90000 lines
processed 100000 lines
skipped 0 empty lines
filtered 0 lines
processed 10000 lines
processed 20000 lines
processed 30000 lines
processed 40000 lines
processed 50000 lines
processed 60000 lines
processed 70000 lines
processed 80000 lines
processed 90000 lines
processed 100000 lines
skipped 0 empty lines
filtered 0 lines
skipped 0 empty lines
filtered 0 lines
skipped 0 empty lines
filtered 0 lines


In [ ]:
# length ratio cleaning
!perl clean-corpus-n.perl --ratio 3 train mhr ru spm_encode/train 1 250
!perl clean-corpus-n.perl --ratio 3 val mhr ru spm_encode/val 1 250

clean-corpus.perl: processing train.mhr & .ru to spm_encode/train, cutoff 1-250, ratio 3
..........(100000)
Input sentences: 108879  Output sentences:  108550
clean-corpus.perl: processing val.mhr & .ru to spm_encode/val, cutoff 1-250, ratio 3

Input sentences: 218  Output sentences:  218


## Binarize

###Eevenki

In [ ]:
#Binarize tokenized text
!fairseq-preprocess \
    --source-lang evn --target-lang ru \
    --trainpref spm_encode/train \
    --validpref spm_encode/val \
    --thresholdsrc 0 --thresholdtgt 0 \
    --destdir data_bin \
    --srcdict data_dict.128k.txt --tgtdict data_dict.128k.txt

/usr/local/lib/python3.8/dist-packages/torch/cuda/__init__.py:497: UserWarning: Can't initialize NVML
  warnings.warn("Can't initialize NVML")
2022-12-16 07:26:33 | INFO | fairseq_cli.preprocess | Namespace(aim_repo=None, aim_run_hash=None, align_suffix=None, alignfile=None, all_gather_list_size=16384, amp=False, amp_batch_retries=2, amp_init_scale=128, amp_scale_window=None, azureml_logging=False, bf16=False, bpe=None, cpu=False, criterion='cross_entropy', dataset_impl='mmap', destdir='data_bin', dict_only=False, empty_cache_freq=0, fp16=False, fp16_init_scale=128, fp16_no_flatten_grads=False, fp16_scale_tolerance=0.0, fp16_scale_window=None, joined_dictionary=False, log_file=None, log_format=None, log_interval=100, lr_scheduler='fixed', memory_efficient_bf16=False, memory_efficient_fp16=False, min_loss_scale=0.0001, model_parallel_size=1, no_progress_bar=False, nwordssrc=-1, nwordstgt=-1, on_cpu_convert_precision=False, only_source=False, optimizer=None, padding_factor=8, plasma_path

In [ ]:
#Binarize tokenized text
!fairseq-preprocess \
    --source-lang evn --target-lang ru \
    --trainpref spm_encode/train_s \
    --validpref spm_encode/val_s \
    --thresholdsrc 0 --thresholdtgt 0 \
    --destdir data_bin \
    --srcdict data_dict.128k.txt --tgtdict data_dict.128k.txt

/usr/local/lib/python3.8/dist-packages/torch/cuda/__init__.py:497: UserWarning: Can't initialize NVML
  warnings.warn("Can't initialize NVML")
2022-12-12 13:13:19 | INFO | fairseq_cli.preprocess | Namespace(aim_repo=None, aim_run_hash=None, align_suffix=None, alignfile=None, all_gather_list_size=16384, amp=False, amp_batch_retries=2, amp_init_scale=128, amp_scale_window=None, azureml_logging=False, bf16=False, bpe=None, cpu=False, criterion='cross_entropy', dataset_impl='mmap', destdir='data_bin', dict_only=False, empty_cache_freq=0, fp16=False, fp16_init_scale=128, fp16_no_flatten_grads=False, fp16_scale_tolerance=0.0, fp16_scale_window=None, joined_dictionary=False, log_file=None, log_format=None, log_interval=100, lr_scheduler='fixed', memory_efficient_bf16=False, memory_efficient_fp16=False, min_loss_scale=0.0001, model_parallel_size=1, no_progress_bar=False, nwordssrc=-1, nwordstgt=-1, on_cpu_convert_precision=False, only_source=False, optimizer=None, padding_factor=8, plasma_path

###Kamas

In [ ]:
#Binarize tokenized text
!fairseq-preprocess \
    --source-lang xas --target-lang ru \
    --trainpref spm_encode/train \
    --validpref spm_encode/val \
    --thresholdsrc 0 --thresholdtgt 0 \
    --destdir data_bin \
    --srcdict data_dict.128k.txt --tgtdict data_dict.128k.txt

/usr/local/lib/python3.8/dist-packages/torch/cuda/__init__.py:497: UserWarning: Can't initialize NVML
  warnings.warn("Can't initialize NVML")
2022-12-16 09:06:21 | INFO | fairseq_cli.preprocess | Namespace(aim_repo=None, aim_run_hash=None, align_suffix=None, alignfile=None, all_gather_list_size=16384, amp=False, amp_batch_retries=2, amp_init_scale=128, amp_scale_window=None, azureml_logging=False, bf16=False, bpe=None, cpu=False, criterion='cross_entropy', dataset_impl='mmap', destdir='data_bin', dict_only=False, empty_cache_freq=0, fp16=False, fp16_init_scale=128, fp16_no_flatten_grads=False, fp16_scale_tolerance=0.0, fp16_scale_window=None, joined_dictionary=False, log_file=None, log_format=None, log_interval=100, lr_scheduler='fixed', memory_efficient_bf16=False, memory_efficient_fp16=False, min_loss_scale=0.0001, model_parallel_size=1, no_progress_bar=False, nwordssrc=-1, nwordstgt=-1, on_cpu_convert_precision=False, only_source=False, optimizer=None, padding_factor=8, plasma_path

###Mari

In [ ]:
#Binarize tokenized text
!fairseq-preprocess \
    --source-lang mhr --target-lang ru \
    --trainpref spm_encode/train \
    --validpref spm_encode/val \
    --thresholdsrc 0 --thresholdtgt 0 \
    --destdir data_bin \
    --srcdict data_dict.128k.txt --tgtdict data_dict.128k.txt

/usr/local/lib/python3.8/dist-packages/torch/cuda/__init__.py:497: UserWarning: Can't initialize NVML
  warnings.warn("Can't initialize NVML")
2022-12-16 17:06:57 | INFO | fairseq_cli.preprocess | Namespace(aim_repo=None, aim_run_hash=None, align_suffix=None, alignfile=None, all_gather_list_size=16384, amp=False, amp_batch_retries=2, amp_init_scale=128, amp_scale_window=None, azureml_logging=False, bf16=False, bpe=None, cpu=False, criterion='cross_entropy', dataset_impl='mmap', destdir='data_bin', dict_only=False, empty_cache_freq=0, fp16=False, fp16_init_scale=128, fp16_no_flatten_grads=False, fp16_scale_tolerance=0.0, fp16_scale_window=None, joined_dictionary=False, log_file=None, log_format=None, log_interval=100, lr_scheduler='fixed', memory_efficient_bf16=False, memory_efficient_fp16=False, min_loss_scale=0.0001, model_parallel_size=1, no_progress_bar=False, nwordssrc=-1, nwordstgt=-1, on_cpu_convert_precision=False, only_source=False, optimizer=None, padding_factor=8, plasma_path

In [ ]:
#Store checkpoints
!mkdir checkpoint

## XAS raining

In [13]:
!fairseq-train data_bin \
  --finetune-from-model  "418M_last_checkpoint.pt"\
  --save-dir checkpoint \
  --task translation_multi_simple_epoch \
  --encoder-normalize-before \
  --langs 'af,am,ar,ast,az,ba,be,bg,bn,br,bs,ca,ceb,cs,cy,da,de,el,en,es,et,fa,ff,fi,fr,fy,ga,gd,gl,gu,ha,he,hi,hr,ht,hu,hy,id,ig,ilo,is,it,ja,jv,ka,kk,km,kn,ko,lb,lg,ln,lo,lt,lv,mg,mk,ml,mn,mr,ms,my,ne,nl,no,ns,oc,or,pa,pl,ps,pt,ro,ru,sd,si,sk,sl,so,sq,sr,ss,su,sv,sw,ta,th,tl,tn,tr,uk,ur,uz,vi,wo,xh,yi,yo,zh,evn,xas,mhr' \
  --lang-pairs lang-pairs.txt \
  --batch-size 64 \
  --decoder-normalize-before \
  --encoder-langtok "src" \
  --decoder-langtok \
  --criterion label_smoothed_cross_entropy --label-smoothing 0.2 \
  --optimizer adam --adam-eps 1e-06 --adam-betas '(0.9, 0.98)' \
  --lr-scheduler inverse_sqrt --lr 3e-05 --warmup-updates 2500 --max-update 400000 \
  --dropout 0.3 --attention-dropout 0.1 --weight-decay 0.0 \
  --update-freq 2 \
  --save-interval 3 \
  --keep-interval-updates 10 \
  --no-epoch-checkpoints \
  --seed 222 \
  --log-format simple \
  --log-interval 4 \
  --arch transformer_wmt_en_de_big \
  --encoder-layers 12 --decoder-layers 12 \
  --encoder-layerdrop 0.05 --decoder-layerdrop 0.05 \
  --share-decoder-input-output-embed --share-all-embeddings

2022-12-18 07:10:21 | INFO | fairseq_cli.train | {'_name': None, 'common': {'_name': None, 'no_progress_bar': False, 'log_interval': 4, 'log_format': 'simple', 'log_file': None, 'aim_repo': None, 'aim_run_hash': None, 'tensorboard_logdir': None, 'wandb_project': None, 'azureml_logging': False, 'seed': 222, 'cpu': False, 'tpu': False, 'bf16': False, 'memory_efficient_bf16': False, 'fp16': False, 'memory_efficient_fp16': False, 'fp16_no_flatten_grads': False, 'fp16_init_scale': 128, 'fp16_scale_window': None, 'fp16_scale_tolerance': 0.0, 'on_cpu_convert_precision': False, 'min_loss_scale': 0.0001, 'threshold_loss_scale': None, 'amp': False, 'amp_batch_retries': 2, 'amp_init_scale': 128, 'amp_scale_window': None, 'user_dir': None, 'empty_cache_freq': 0, 'all_gather_list_size': 16384, 'model_parallel_size': 1, 'quantization_config_path': None, 'profile': False, 'reset_logging': False, 'suppress_crashes': False, 'use_plasma_view': False, 'plasma_path': '/tmp/plasma'}, 'common_eval': {'_name

In [ ]:
!fairseq-train data_bin \
  --finetune-from-model  "418M_last_checkpoint.pt"\
  --save-dir checkpoint \
  --task translation_multi_simple_epoch \
  --encoder-normalize-before \
  --langs 'af,am,ar,ast,az,ba,be,bg,bn,br,bs,ca,ceb,cs,cy,da,de,el,en,es,et,fa,ff,fi,fr,fy,ga,gd,gl,gu,ha,he,hi,hr,ht,hu,hy,id,ig,ilo,is,it,ja,jv,ka,kk,km,kn,ko,lb,lg,ln,lo,lt,lv,mg,mk,ml,mn,mr,ms,my,ne,nl,no,ns,oc,or,pa,pl,ps,pt,ro,ru,sd,si,sk,sl,so,sq,sr,ss,su,sv,sw,ta,th,tl,tn,tr,uk,ur,uz,vi,wo,xh,yi,yo,zh,zu,mhr' \
  --lang-pairs 'mhr-ru' \
  --batch-size 16 \
  --decoder-normalize-before \
  --encoder-langtok "src" \
  --decoder-langtok \
  --criterion label_smoothed_cross_entropy --label-smoothing 0.2 \
  --optimizer adam --adam-eps 1e-06 --adam-betas '(0.9, 0.98)' \
  --lr-scheduler inverse_sqrt --lr 3e-05 --warmup-updates 2500 --max-update 40000 \
  --dropout 0.3 --attention-dropout 0.1 --weight-decay 0.0 \
  --update-freq 2 \
  --save-interval 1 \
  --save-interval-updates 5000 \
  --keep-interval-updates 10 \
  --no-epoch-checkpoints \
  --seed 222 \
  --log-format simple \
  --log-interval 4 \
  --arch transformer_wmt_en_de_big \
  --encoder-layers 12 --decoder-layers 12 \
  --encoder-layerdrop 0.05 --decoder-layerdrop 0.05 \
  --share-decoder-input-output-embed --share-all-embeddings

2022-12-16 17:48:48 | INFO | fairseq_cli.train | {'_name': None, 'common': {'_name': None, 'no_progress_bar': False, 'log_interval': 4, 'log_format': 'simple', 'log_file': None, 'aim_repo': None, 'aim_run_hash': None, 'tensorboard_logdir': None, 'wandb_project': None, 'azureml_logging': False, 'seed': 222, 'cpu': False, 'tpu': False, 'bf16': False, 'memory_efficient_bf16': False, 'fp16': False, 'memory_efficient_fp16': False, 'fp16_no_flatten_grads': False, 'fp16_init_scale': 128, 'fp16_scale_window': None, 'fp16_scale_tolerance': 0.0, 'on_cpu_convert_precision': False, 'min_loss_scale': 0.0001, 'threshold_loss_scale': None, 'amp': False, 'amp_batch_retries': 2, 'amp_init_scale': 128, 'amp_scale_window': None, 'user_dir': None, 'empty_cache_freq': 0, 'all_gather_list_size': 16384, 'model_parallel_size': 1, 'quantization_config_path': None, 'profile': False, 'reset_logging': False, 'suppress_crashes': False, 'use_plasma_view': False, 'plasma_path': '/tmp/plasma'}, 'common_eval': {'_name

In [ ]:
!fairseq-train data_bin \
  --finetune-from-model  "418M_last_checkpoint.pt"\
  --save-dir checkpoint \
  --task translation_multi_simple_epoch \
  --encoder-normalize-before \
  --langs 'af,am,ar,ast,az,ba,be,bg,bn,br,bs,ca,ceb,cs,cy,da,de,el,en,es,et,fa,ff,fi,fr,fy,ga,gd,gl,gu,ha,he,hi,hr,ht,hu,hy,id,ig,ilo,is,it,ja,jv,ka,kk,km,kn,ko,lb,lg,ln,lo,lt,lv,mg,mk,ml,mn,mr,ms,my,ne,nl,no,ns,oc,or,pa,pl,ps,pt,ro,ru,sd,si,sk,sl,so,sq,sr,ss,su,sv,sw,ta,th,tl,tn,tr,uk,ur,uz,vi,wo,xh,yi,yo,zh,zu,evn' \
  --lang-pairs 'evn-ru' \
  --batch-size 10 \
  --decoder-normalize-before \
  --encoder-langtok "src" \
  --decoder-langtok \
  --criterion cross_entropy \
  --optimizer adafactor \
  --lr-scheduler cosine --lr 3e-05 --warmup-updates 2500 --max-update 40000 \
  --dropout 0.3 --attention-dropout 0.1 --weight-decay 0.0 \
  --update-freq 2 \
  --save-interval 4 \
  --save-interval-updates 5000 \
  --keep-interval-updates 10 \
  --no-epoch-checkpoints \
  --seed 222 \
  --log-format simple \
  --log-interval 4 \
  --arch transformer_wmt_en_de_big \
  --encoder-layers 12 --decoder-layers 12 \
  --encoder-layerdrop 0.05 --decoder-layerdrop 0.05 \
  --ddp-backend no_c10d \
  --share-decoder-input-output-embed --share-all-embeddings

2022-12-14 15:39:12 | INFO | fairseq_cli.train | {'_name': None, 'common': {'_name': None, 'no_progress_bar': False, 'log_interval': 4, 'log_format': 'simple', 'log_file': None, 'aim_repo': None, 'aim_run_hash': None, 'tensorboard_logdir': None, 'wandb_project': None, 'azureml_logging': False, 'seed': 222, 'cpu': False, 'tpu': False, 'bf16': False, 'memory_efficient_bf16': False, 'fp16': False, 'memory_efficient_fp16': False, 'fp16_no_flatten_grads': False, 'fp16_init_scale': 128, 'fp16_scale_window': None, 'fp16_scale_tolerance': 0.0, 'on_cpu_convert_precision': False, 'min_loss_scale': 0.0001, 'threshold_loss_scale': None, 'amp': False, 'amp_batch_retries': 2, 'amp_init_scale': 128, 'amp_scale_window': None, 'user_dir': None, 'empty_cache_freq': 0, 'all_gather_list_size': 16384, 'model_parallel_size': 1, 'quantization_config_path': None, 'profile': False, 'reset_logging': False, 'suppress_crashes': False, 'use_plasma_view': False, 'plasma_path': '/tmp/plasma'}, 'common_eval': {'_name

## EVN training

In [ ]:
af,am,ar,ast,az,ba,be,bg,bn,br,bs,ca,ceb,cs,cy,da,de,el,en,es,et,fa,ff,fi,fr,fy,ga,gd,gl,gu,ha,he,hi,hr,ht,hu,hy,id,ig,ilo,is,it,ja,jv,ka,kk,km,kn,ko,lb,lg,ln,lo,lt,lv,mg,mk,ml,mn,mr,ms,my,ne,nl,no,ns,oc,or,pa,pl,ps,pt,ro,ru,sd,si,sk,sl,so,sq,sr,ss,su,sv,sw,ta,th,tl,tn,tr,uk,ur,uz,vi,wo,xh,yi,yo,zh,zu'

In [ ]:
!fairseq-train data_bin \
  --finetune-from-model  "418M_last_checkpoint.pt"\
  --save-dir checkpoint \
  --task translation_multi_simple_epoch \
  --encoder-normalize-before \
  --langs 'aav,aed,af,alv,am,ar,art,ase,ast,az,ba,bat,bcl,be,bg,bn,br,bs,ca,ceb,cs,cy,da,de,el,en,es,et,fa,ff,fi,fr,fy,ga,gd,gl,gu,ha,he,hi,hr,ht,hu,hy,id,ig,ilo,is,it,ja,jv,ka,kk,km,kn,ko,lb,lg,ln,lo,lt,lv,mg,mk,ml,mn,mr,ms,my,ne,nl,no,ns,oc,or,pa,pl,ps,pt,ro,ru,sd,si,sk,sl,so,sq,sr,ss,su,sv,sw,ta,th,tl,tn,tr,uk,ur,uz,vi,wo,xh,yi,yo,zh,zu,evn' \
  --lang-pairs 'evn-ru' \
  --batch-size 10 \
  --decoder-normalize-before \
  --encoder-langtok "src" \
  --decoder-langtok \
  --criterion label_smoothed_cross_entropy --label-smoothing 0.2 \
  --optimizer adam --adam-eps 1e-06 --adam-betas '(0.9, 0.98)' \
  --lr-scheduler inverse_sqrt --lr 3e-05 --warmup-updates 2500 --max-update 40000 \
  --dropout 0.3 --attention-dropout 0.1 --weight-decay 0.0 \
  --update-freq 2 \
  --save-interval 3 \
  --save-interval-updates 5000 \
  --keep-interval-updates 10 \
  --no-epoch-checkpoints \
  --seed 222 \
  --log-format simple \
  --log-interval 4 \
  --arch transformer_wmt_en_de_big \
  --encoder-layers 12 --decoder-layers 12 \
  --encoder-layerdrop 0.05 --decoder-layerdrop 0.05 \
  --share-decoder-input-output-embed --share-all-embeddings

2022-12-16 02:42:10 | INFO | fairseq_cli.train | {'_name': None, 'common': {'_name': None, 'no_progress_bar': False, 'log_interval': 4, 'log_format': 'simple', 'log_file': None, 'aim_repo': None, 'aim_run_hash': None, 'tensorboard_logdir': None, 'wandb_project': None, 'azureml_logging': False, 'seed': 222, 'cpu': False, 'tpu': False, 'bf16': False, 'memory_efficient_bf16': False, 'fp16': False, 'memory_efficient_fp16': False, 'fp16_no_flatten_grads': False, 'fp16_init_scale': 128, 'fp16_scale_window': None, 'fp16_scale_tolerance': 0.0, 'on_cpu_convert_precision': False, 'min_loss_scale': 0.0001, 'threshold_loss_scale': None, 'amp': False, 'amp_batch_retries': 2, 'amp_init_scale': 128, 'amp_scale_window': None, 'user_dir': None, 'empty_cache_freq': 0, 'all_gather_list_size': 16384, 'model_parallel_size': 1, 'quantization_config_path': None, 'profile': False, 'reset_logging': False, 'suppress_crashes': False, 'use_plasma_view': False, 'plasma_path': '/tmp/plasma'}, 'common_eval': {'_name

In [ ]:
!fairseq-train data_bin \
  --finetune-from-model  "1.2B_last_checkpoint.pt"\
  --save-dir checkpoint \
  --task translation_multi_simple_epoch \
  --sampling-method "temperature" \
  --sampling-temperature 1.5 \
  --encoder-normalize-before \
  --langs 'aav,aed,af,alv,am,ar,art,ase,ast,az,ba,bat,bcl,be,bg,bn,br,bs,ca,ceb,cs,cy,da,de,el,en,es,et,fa,ff,fi,fr,fy,ga,gd,gl,gu,ha,he,hi,hr,ht,hu,hy,id,ig,ilo,is,it,ja,jv,ka,kk,km,kn,ko,lb,lg,ln,lo,lt,lv,mg,mk,ml,mn,mr,ms,my,ne,nl,no,ns,oc,or,pa,pl,ps,pt,ro,ru,sd,si,sk,sl,so,sq,sr,ss,su,sv,sw,ta,th,tl,tn,tr,uk,ur,uz,vi,wo,xh,yi,yo,zh,zu,evn' \
  --lang-pairs 'evn-ru' \
  --max-tokens 1024 \
  --decoder-normalize-before \
  --encoder-langtok "src" \
  --decoder-langtok \
  --criterion label_smoothed_cross_entropy --label-smoothing 0.2 \
  --optimizer adam --adam-eps 1e-06 --adam-betas '(0.9, 0.98)' \
  --lr-scheduler inverse_sqrt --lr 3e-05 --warmup-updates 2500 --max-update 40000 \
  --dropout 0.3 --attention-dropout 0.1 --weight-decay 0.0 \
  --update-freq 2 \
  --save-interval 10 \
  --save-interval-updates 5000 \
  --keep-interval-updates 10 \
  --no-epoch-checkpoints \
  --seed 222 \
  --log-format simple \
  --log-interval 4 \
  --arch transformer_wmt_en_de_big \
  --encoder-layers 24 --decoder-layers 24 \
  --encoder-ffn-embed-dim 8192 --decoder-ffn-embed-dim 8192 \
  --share-decoder-input-output-embed --share-all-embeddings

2022-12-06 10:41:38 | INFO | fairseq_cli.train | {'_name': None, 'common': {'_name': None, 'no_progress_bar': False, 'log_interval': 4, 'log_format': 'simple', 'log_file': None, 'aim_repo': None, 'aim_run_hash': None, 'tensorboard_logdir': None, 'wandb_project': None, 'azureml_logging': False, 'seed': 222, 'cpu': False, 'tpu': False, 'bf16': False, 'memory_efficient_bf16': False, 'fp16': False, 'memory_efficient_fp16': False, 'fp16_no_flatten_grads': False, 'fp16_init_scale': 128, 'fp16_scale_window': None, 'fp16_scale_tolerance': 0.0, 'on_cpu_convert_precision': False, 'min_loss_scale': 0.0001, 'threshold_loss_scale': None, 'amp': False, 'amp_batch_retries': 2, 'amp_init_scale': 128, 'amp_scale_window': None, 'user_dir': None, 'empty_cache_freq': 0, 'all_gather_list_size': 16384, 'model_parallel_size': 1, 'quantization_config_path': None, 'profile': False, 'reset_logging': False, 'suppress_crashes': False, 'use_plasma_view': False, 'plasma_path': '/tmp/plasma'}, 'common_eval': {'_name

## Read test file

In [89]:
new_test = pd.read_csv(os.path.join(PATH_TO_DATASET, 'evn_test.txt'),header=None,sep="\t")

In [22]:
# new_test = pd.read_csv(os.path.join(PATH_TO_DATASET, 'evn_test_s.txt'),header=None,sep="\t")

In [44]:
new_test = pd.read_csv(os.path.join(PATH_TO_DATASET, 'mhr_test.txt'),header=None,sep="\t")

In [101]:
new_test = pd.read_csv(os.path.join(PATH_TO_DATASET, 'xas_test.txt'),header=None,sep="\t")

In [32]:
test_txt = "\n".join(new_test[0].tolist())

In [26]:
file = open("evenki_txt.txt", "w")
file.write(test_txt)
file.close()

In [12]:
file = open("mari_txt.txt", "w")
file.write(test_txt)
file.close()

In [33]:
file = open("xas_txt.txt", "w")
file.write(test_txt)
file.close()

## Model inference

### Evenki

In [27]:
!python ../low_resource_mt-main/fairseq/scripts/spm_encode.py \
        --model spm.128k.model \
        --output_format=piece \
        --inputs=evenki_txt.txt \
        --outputs=evenki_txt.evn

skipped 0 empty lines
filtered 0 lines


In [28]:
!fairseq-preprocess \
    --source-lang evn --target-lang ru \
    --only-source	\
    --testpref evenki_txt \
    --thresholdsrc 0 --thresholdtgt 0 \
    --destdir data_bin \
    --srcdict data_dict.128k.txt --tgtdict data_dict.128k.txt  

/usr/local/lib/python3.8/dist-packages/torch/cuda/__init__.py:497: UserWarning: Can't initialize NVML
  warnings.warn("Can't initialize NVML")
2022-12-18 06:39:36 | INFO | fairseq_cli.preprocess | Namespace(aim_repo=None, aim_run_hash=None, align_suffix=None, alignfile=None, all_gather_list_size=16384, amp=False, amp_batch_retries=2, amp_init_scale=128, amp_scale_window=None, azureml_logging=False, bf16=False, bpe=None, cpu=False, criterion='cross_entropy', dataset_impl='mmap', destdir='data_bin', dict_only=False, empty_cache_freq=0, fp16=False, fp16_init_scale=128, fp16_no_flatten_grads=False, fp16_scale_tolerance=0.0, fp16_scale_window=None, joined_dictionary=False, log_file=None, log_format=None, log_interval=100, lr_scheduler='fixed', memory_efficient_bf16=False, memory_efficient_fp16=False, min_loss_scale=0.0001, model_parallel_size=1, no_progress_bar=False, nwordssrc=-1, nwordstgt=-1, on_cpu_convert_precision=False, only_source=True, optimizer=None, padding_factor=8, plasma_path=

In [85]:
!fairseq-generate "data_bin/"  --batch-size 32 \
      --path "checkpoint/checkpoint_best.pt.evn-xas.pt" \
      --fixed-dictionary model_dict.128k.txt \
      -s evn -t ru \
      --remove-bpe 'sentencepiece' \
      --beam 5 \
      --task translation_multi_simple_epoch \
      --lang-pairs language_pairs_small_models.txt \
      --decoder-langtok \
      --encoder-langtok tgt \
      --gen-subset test > gen_out

/usr/local/lib/python3.8/dist-packages/torch/cuda/__init__.py:497: UserWarning: Can't initialize NVML
  warnings.warn("Can't initialize NVML")
2022-12-18 10:55:14 | INFO | fairseq_cli.generate | {'_name': None, 'common': {'_name': None, 'no_progress_bar': False, 'log_interval': 100, 'log_format': None, 'log_file': None, 'aim_repo': None, 'aim_run_hash': None, 'tensorboard_logdir': None, 'wandb_project': None, 'azureml_logging': False, 'seed': 1, 'cpu': False, 'tpu': False, 'bf16': False, 'memory_efficient_bf16': False, 'fp16': False, 'memory_efficient_fp16': False, 'fp16_no_flatten_grads': False, 'fp16_init_scale': 128, 'fp16_scale_window': None, 'fp16_scale_tolerance': 0.0, 'on_cpu_convert_precision': False, 'min_loss_scale': 0.0001, 'threshold_loss_scale': None, 'amp': False, 'amp_batch_retries': 2, 'amp_init_scale': 128, 'amp_scale_window': None, 'user_dir': None, 'empty_cache_freq': 0, 'all_gather_list_size': 16384, 'model_parallel_size': 1, 'quantization_config_path': None, 'profi

### Kamas

In [34]:
!python ../low_resource_mt-main/fairseq/scripts/spm_encode.py \
        --model spm.128k.model \
        --output_format=piece \
        --inputs=xas_txt.txt \
        --outputs=xas_txt.xas

skipped 0 empty lines
filtered 0 lines


In [35]:
!fairseq-preprocess \
    --source-lang xas --target-lang ru \
    --only-source	\
    --testpref xas_txt \
    --thresholdsrc 0 --thresholdtgt 0 \
    --destdir data_bin \
    --srcdict xas_data_dict.128k.txt --tgtdict xas_data_dict.128k.txt  

/usr/local/lib/python3.8/dist-packages/torch/cuda/__init__.py:497: UserWarning: Can't initialize NVML
  warnings.warn("Can't initialize NVML")
2022-12-18 09:42:27 | INFO | fairseq_cli.preprocess | Namespace(aim_repo=None, aim_run_hash=None, align_suffix=None, alignfile=None, all_gather_list_size=16384, amp=False, amp_batch_retries=2, amp_init_scale=128, amp_scale_window=None, azureml_logging=False, bf16=False, bpe=None, cpu=False, criterion='cross_entropy', dataset_impl='mmap', destdir='data_bin', dict_only=False, empty_cache_freq=0, fp16=False, fp16_init_scale=128, fp16_no_flatten_grads=False, fp16_scale_tolerance=0.0, fp16_scale_window=None, joined_dictionary=False, log_file=None, log_format=None, log_interval=100, lr_scheduler='fixed', memory_efficient_bf16=False, memory_efficient_fp16=False, min_loss_scale=0.0001, model_parallel_size=1, no_progress_bar=False, nwordssrc=-1, nwordstgt=-1, on_cpu_convert_precision=False, only_source=True, optimizer=None, padding_factor=8, plasma_path=

In [ ]:
!fairseq-generate "data_bin/"  --batch-size 32 \
      --path "checkpoint/checkpoint_best.pt" \
      --fixed-dictionary xas_model_dict.128k.txt \
      -s evn -t ru \
      --remove-bpe 'sentencepiece' \
      --beam 5 \
      --task translation_multi_simple_epoch \
      --lang-pairs language_pairs_small_models.txt \
      --decoder-langtok \
      --encoder-langtok tgt \
      --gen-subset test > gen_out

/usr/local/lib/python3.8/dist-packages/torch/cuda/__init__.py:497: UserWarning: Can't initialize NVML
  warnings.warn("Can't initialize NVML")
2022-12-16 12:55:02 | INFO | fairseq_cli.generate | {'_name': None, 'common': {'_name': None, 'no_progress_bar': False, 'log_interval': 100, 'log_format': None, 'log_file': None, 'aim_repo': None, 'aim_run_hash': None, 'tensorboard_logdir': None, 'wandb_project': None, 'azureml_logging': False, 'seed': 1, 'cpu': False, 'tpu': False, 'bf16': False, 'memory_efficient_bf16': False, 'fp16': False, 'memory_efficient_fp16': False, 'fp16_no_flatten_grads': False, 'fp16_init_scale': 128, 'fp16_scale_window': None, 'fp16_scale_tolerance': 0.0, 'on_cpu_convert_precision': False, 'min_loss_scale': 0.0001, 'threshold_loss_scale': None, 'amp': False, 'amp_batch_retries': 2, 'amp_init_scale': 128, 'amp_scale_window': None, 'user_dir': None, 'empty_cache_freq': 0, 'all_gather_list_size': 16384, 'model_parallel_size': 1, 'quantization_config_path': None, 'profi

In [100]:
!fairseq-generate "data_bin/"  --batch-size 32 \
      --path "checkpoint/checkpoint_best.pt.evn-xas.pt" \
      --fixed-dictionary xas_model_dict.128k.txt \
      -s xas -t ru \
      --remove-bpe 'sentencepiece' \
      --beam 5 \
      --task translation_multi_simple_epoch \
      --lang-pairs language_pairs_small_models.txt \
      --decoder-langtok \
      --encoder-langtok tgt \
      --gen-subset test > gen_out

/usr/local/lib/python3.8/dist-packages/torch/cuda/__init__.py:497: UserWarning: Can't initialize NVML
  warnings.warn("Can't initialize NVML")
2022-12-18 11:22:41 | INFO | fairseq_cli.generate | {'_name': None, 'common': {'_name': None, 'no_progress_bar': False, 'log_interval': 100, 'log_format': None, 'log_file': None, 'aim_repo': None, 'aim_run_hash': None, 'tensorboard_logdir': None, 'wandb_project': None, 'azureml_logging': False, 'seed': 1, 'cpu': False, 'tpu': False, 'bf16': False, 'memory_efficient_bf16': False, 'fp16': False, 'memory_efficient_fp16': False, 'fp16_no_flatten_grads': False, 'fp16_init_scale': 128, 'fp16_scale_window': None, 'fp16_scale_tolerance': 0.0, 'on_cpu_convert_precision': False, 'min_loss_scale': 0.0001, 'threshold_loss_scale': None, 'amp': False, 'amp_batch_retries': 2, 'amp_init_scale': 128, 'amp_scale_window': None, 'user_dir': None, 'empty_cache_freq': 0, 'all_gather_list_size': 16384, 'model_parallel_size': 1, 'quantization_config_path': None, 'profi

### Mari

In [13]:
!python ../low_resource_mt-main/fairseq/scripts/spm_encode.py \
        --model spm.128k.model \
        --output_format=piece \
        --inputs=mari_txt.txt \
        --outputs=mari_txt.mhr

skipped 0 empty lines
filtered 0 lines


In [14]:
!fairseq-preprocess \
    --source-lang mhr --target-lang ru \
    --only-source	\
    --testpref mari_txt \
    --thresholdsrc 0 --thresholdtgt 0 \
    --destdir data_bin \
    --srcdict data_dict.128k.txt --tgtdict data_dict.128k.txt  

/usr/local/lib/python3.8/dist-packages/torch/cuda/__init__.py:497: UserWarning: Can't initialize NVML
  warnings.warn("Can't initialize NVML")
2022-12-18 09:29:29 | INFO | fairseq_cli.preprocess | Namespace(aim_repo=None, aim_run_hash=None, align_suffix=None, alignfile=None, all_gather_list_size=16384, amp=False, amp_batch_retries=2, amp_init_scale=128, amp_scale_window=None, azureml_logging=False, bf16=False, bpe=None, cpu=False, criterion='cross_entropy', dataset_impl='mmap', destdir='data_bin', dict_only=False, empty_cache_freq=0, fp16=False, fp16_init_scale=128, fp16_no_flatten_grads=False, fp16_scale_tolerance=0.0, fp16_scale_window=None, joined_dictionary=False, log_file=None, log_format=None, log_interval=100, lr_scheduler='fixed', memory_efficient_bf16=False, memory_efficient_fp16=False, min_loss_scale=0.0001, model_parallel_size=1, no_progress_bar=False, nwordssrc=-1, nwordstgt=-1, on_cpu_convert_precision=False, only_source=True, optimizer=None, padding_factor=8, plasma_path=

In [52]:
!fairseq-generate "data_bin/"  --batch-size 32 \
      --path "checkpoint/checkpoint_best.pt" \
      --fixed-dictionary model_dict.128k.txt \
      -s mhr -t ru \
      --remove-bpe 'sentencepiece' \
      --beam 5 \
      --task translation_multi_simple_epoch \
      --lang-pairs mhr_language_pairs_small_models.txt \
      --decoder-langtok \
      --encoder-langtok tgt \
      --gen-subset test > gen_out

/usr/local/lib/python3.8/dist-packages/torch/cuda/__init__.py:497: UserWarning: Can't initialize NVML
  warnings.warn("Can't initialize NVML")
2022-12-18 10:01:31 | INFO | fairseq_cli.generate | {'_name': None, 'common': {'_name': None, 'no_progress_bar': False, 'log_interval': 100, 'log_format': None, 'log_file': None, 'aim_repo': None, 'aim_run_hash': None, 'tensorboard_logdir': None, 'wandb_project': None, 'azureml_logging': False, 'seed': 1, 'cpu': False, 'tpu': False, 'bf16': False, 'memory_efficient_bf16': False, 'fp16': False, 'memory_efficient_fp16': False, 'fp16_no_flatten_grads': False, 'fp16_init_scale': 128, 'fp16_scale_window': None, 'fp16_scale_tolerance': 0.0, 'on_cpu_convert_precision': False, 'min_loss_scale': 0.0001, 'threshold_loss_scale': None, 'amp': False, 'amp_batch_retries': 2, 'amp_init_scale': 128, 'amp_scale_window': None, 'user_dir': None, 'empty_cache_freq': 0, 'all_gather_list_size': 16384, 'model_parallel_size': 1, 'quantization_config_path': None, 'profi

### Evaluation

In [16]:
from nltk.translate import bleu_score

def compute_4_bleu(refs, hyps):
    bleu1 = bleu_score.corpus_bleu(refs, hyps, weights=(1,))
    bleu2 = bleu_score.corpus_bleu(refs, hyps, weights=(0.5, 0.5))
    bleu3 = bleu_score.corpus_bleu(refs, hyps, weights=(1/3, 1/3, 1/3))
    bleu4 = bleu_score.corpus_bleu(refs, hyps, weights=(0.25, 0.25, 0.25, 0.25))
    for num, bleu in enumerate((bleu1, bleu2, bleu3, bleu4)):
        print(f'BLEU{num+1} = {round(bleu*100, 1)}/100')
    return bleu1*100, bleu2*100, bleu3*100, bleu4*100

In [86]:
!cat gen_out | grep -P "^H" |sort -V |cut -f 3- | sed 's/\[ru\]//g' |sacremoses tokenize > ev_ru.hyp

In [87]:
with open("ev_ru.hyp") as hyp:
  hyps = [line for line in hyp]

In [90]:
result = compute_4_bleu(new_test[1].tolist(), hyps)

/usr/local/lib/python3.8/dist-packages/nltk/translate/bleu_score.py:552: UserWarning: 
The hypothesis contains 0 counts of 2-gram overlaps.
Therefore the BLEU score evaluates to 0, independently of
how many N-gram overlaps of lower order it contains.
Consider using lower n-gram order or use SmoothingFunction()
  warnings.warn(_msg)
/usr/local/lib/python3.8/dist-packages/nltk/translate/bleu_score.py:552: UserWarning: 
The hypothesis contains 0 counts of 3-gram overlaps.
Therefore the BLEU score evaluates to 0, independently of
how many N-gram overlaps of lower order it contains.
Consider using lower n-gram order or use SmoothingFunction()
  warnings.warn(_msg)


BLEU1 = 46.5/100
BLEU2 = 0.0/100
BLEU3 = 0.0/100
BLEU4 = 0.0/100


/usr/local/lib/python3.8/dist-packages/nltk/translate/bleu_score.py:552: UserWarning: 
The hypothesis contains 0 counts of 4-gram overlaps.
Therefore the BLEU score evaluates to 0, independently of
how many N-gram overlaps of lower order it contains.
Consider using lower n-gram order or use SmoothingFunction()
  warnings.warn(_msg)


In [54]:
!cat gen_out | grep -P "^H" |sort -V |cut -f 3- | sed 's/\[ru\]//g' |sacremoses detokenize > mhr_ru.hyp

In [102]:
!cat gen_out | grep -P "^H" |sort -V |cut -f 3- | sed 's/\[ru\]//g' |sacremoses detokenize > xas_ru.hyp

In [55]:
with open("mhr_ru.hyp") as hyp:
  hyps = [line.strip() for line in hyp]

In [103]:
with open("xas_ru.hyp") as hyp:
  hyps = [line.strip() for line in hyp]

In [104]:
result = compute_4_bleu(new_test[1].tolist(), hyps)

/usr/local/lib/python3.8/dist-packages/nltk/translate/bleu_score.py:552: UserWarning: 
The hypothesis contains 0 counts of 2-gram overlaps.
Therefore the BLEU score evaluates to 0, independently of
how many N-gram overlaps of lower order it contains.
Consider using lower n-gram order or use SmoothingFunction()
  warnings.warn(_msg)
/usr/local/lib/python3.8/dist-packages/nltk/translate/bleu_score.py:552: UserWarning: 
The hypothesis contains 0 counts of 3-gram overlaps.
Therefore the BLEU score evaluates to 0, independently of
how many N-gram overlaps of lower order it contains.
Consider using lower n-gram order or use SmoothingFunction()
  warnings.warn(_msg)


BLEU1 = 55.1/100
BLEU2 = 0.0/100
BLEU3 = 0.0/100
BLEU4 = 0.0/100


/usr/local/lib/python3.8/dist-packages/nltk/translate/bleu_score.py:552: UserWarning: 
The hypothesis contains 0 counts of 4-gram overlaps.
Therefore the BLEU score evaluates to 0, independently of
how many N-gram overlaps of lower order it contains.
Consider using lower n-gram order or use SmoothingFunction()
  warnings.warn(_msg)


In [105]:
rus_ref = "\n".join(new_test[1].tolist())
file = open(PATH_TO_DATASET+"/russian_test.ref", "w")
file.write(rus_ref)
file.close()

In [27]:
from sacrebleu import sentence_bleu

In [28]:
import sacrebleu

In [92]:
!sacrebleu -tok 'none' -s 'none' russian_test.ref < ev_ru.hyp

{
 "name": "BLEU",
 "score": 65.7,
 "signature": "nrefs:1|case:mixed|eff:no|tok:none|smooth:none|version:2.3.1",
 "verbose_score": "81.8/70.9/62.9/56.8 (BP = 0.974 ratio = 0.974 hyp_len = 1030 ref_len = 1057)",
 "nrefs": "1",
 "case": "mixed",
 "eff": "no",
 "tok": "none",
 "smooth": "none",
 "version": "2.3.1"
}


In [106]:
!sacrebleu -tok 'none' -s 'none' russian_test.ref < xas_ru.hyp

{
 "name": "BLEU",
 "score": 75.2,
 "signature": "nrefs:1|case:mixed|eff:no|tok:none|smooth:none|version:2.3.1",
 "verbose_score": "86.7/77.3/72.0/68.4 (BP = 0.992 ratio = 0.992 hyp_len = 910 ref_len = 917)",
 "nrefs": "1",
 "case": "mixed",
 "eff": "no",
 "tok": "none",
 "smooth": "none",
 "version": "2.3.1"
}


In [51]:
!sacrebleu -tok 'none' -s 'none' russian_test.ref < mhr_ru.hyp

{
 "name": "BLEU",
 "score": 24.1,
 "signature": "nrefs:1|case:mixed|eff:no|tok:none|smooth:none|version:2.3.1",
 "verbose_score": "56.6/30.2/17.9/11.1 (BP = 1.000 ratio = 1.068 hyp_len = 1000 ref_len = 936)",
 "nrefs": "1",
 "case": "mixed",
 "eff": "no",
 "tok": "none",
 "smooth": "none",
 "version": "2.3.1"
}


In [20]:
!pip install evaluate

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 81 kB 3.5 MB/s 
     |████████████████████████████████| 451 kB 23.8 MB/s 
     |████████████████████████████████| 132 kB 61.4 MB/s 
     |████████████████████████████████| 212 kB 46.0 MB/s 
     |████████████████████████████████| 182 kB 65.3 MB/s 
     |████████████████████████████████| 127 kB 71.6 MB/s 
  Attempting uninstall: urllib3
    Found existing installation: urllib3 1.24.3
    Uninstalling urllib3-1.24.3:
      Successfully uninstalled urllib3-1.24.3


In [21]:
import evaluate

In [22]:
bleu_metric = evaluate.load("bleu")

In [107]:
result = bleu_metric.compute(predictions=hyps, references=new_test[1].tolist())

In [108]:
result

{'bleu': 0.7471668079683464,
 'precisions': [0.8566775244299675,
  0.7609618104667609,
  0.7082494969818913,
  0.675],
 'brevity_penalty': 1.0,
 'length_ratio': 1.0043620501635768,
 'translation_length': 921,
 'reference_length': 917}